<a href="https://colab.research.google.com/github/tlsdmswn01/PyTorch_Study/blob/main/Alexnet%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터 준비

In [ ]:
!gdown --id 1iog5aaxqS_U9sOPPAgdIIly8pxSZ1Cf7

In [ ]:
!unzip ./data.zip

In [ ]:
import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as T
from PIL import Image

from tqdm import tqdm
import numpy as np

In [ ]:
#[[이미지_path],[라벨]] 이렇게 데이터 구성하기
#[['/content/training_set/cats/cat.1.jpg'],[0]]
class CustomDataset(Dataset):
    def __init__(self,root_dir,transform=None):
        self.root_dir=root_dir
        self.transform = transform
        self.classes=os.listdir(root_dir)
        self.data=[]


        for label in range(len(self.classes)):
            class_folder = os.path.join(root_dir,self.classes[label])
            for filename in os.listdir(class_folder):
                img_path=os.path.join(class_folder,filename)
                self.data.append([img_path,label])

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):

        img_path,label=self.data[idx]

        img=Image.open(img_path)

        if self.transform:
            image=self.transform(img)

        return image, label


In [ ]:
# 이미지 사이즈가 다 다름
img=Image.open('/content/training_set/cats/cat.100.jpg')
img.size

In [ ]:
batch_size=32
learning_rate=0.0001

train_transform=T.Compose([
    T.Resize((256,256)),
    T.CenterCrop((227,227)),
    T.RandomHorizontalFlip(0.5),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # 이미지 정규화
])

test_transform=T.Compose([
    T.Resize((227,227)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # 이미지 정규화
])


In [ ]:
train_dataset=CustomDataset('./training_set/',transform=train_transform)
train_loader=DataLoader(train_dataset,batch_size=batch_size, drop_last=True, shuffle=True)
test_dataset=CustomDataset('./test_set/',transform=train_transform)
test_loader=DataLoader(test_dataset,batch_size=batch_size, shuffle=False)

In [ ]:
image,label=next(iter(train_loader))
print(image.size(0),label)

## Alexnet 모델

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
class Alexnet(nn.Module):
    def __init__(self):
        super(Alexnet,self).__init__()


        self.layer=nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=96, kernel_size=(11,11),stride=4),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2),

            nn.Conv2d(in_channels=96,out_channels=256, kernel_size=(5,5),stride=1,padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2),

            nn.Conv2d(in_channels=256,out_channels=384, kernel_size=(3,3),stride=1,padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=384,out_channels=384, kernel_size=(3,3),stride=1,padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=384,out_channels=256, kernel_size=(3,3),stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2),
        )

        self.linear=nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(6*6*256,4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096,4096),
            nn.ReLU(),
            nn.Linear(4096,1000)
        )

    def forward(self,x):
        x=self.layer(x)
        x = x.view(-1, 256*6*6)
        x=self.linear(x)
        return x


In [ ]:
import torchsummary
model=Alexnet()
model.to(device)

torchsummary.summary(model,input_size=(3,227,227),device='cuda')

In [ ]:
optimizer = optim.Adam(model.parameters(),lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
def train(train_loader,model,loss_fn,optimizer,device):

    model.train()

    #Loss와 accuracy 계산 하기
    r_size=0
    r_loss=0
    corr=0

    # 예쁘게 Progress Bar를 출력하면서 훈련 상태를 모니터링 하기 위하여 tqdm으로 래핑합
    progress_bar=tqdm(train_loader)

    for idx, (img,label) in enumerate(progress_bar,start=1):
        img,label= img.to(device), label.to(device)

        optimizer.zero_grad()
        output=model(img)
        loss=loss_fn(output,label)
        loss.backward()
        optimizer.step()

        # output결과값의 확률은 무시하고, 인덱스 값만 가져오기
        _,pred = output.max(dim=1)

        # pred.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산. item()은 tensor에서 값을 추출.
        # 합계는 corr 변수에 누적
        corr+=pred.eq(label).sum().item()

        r_loss+=loss.item()*img.size(0)
        r_size+=img.size(0)
        progress_bar.set_description(f'[Training] loss: {r_loss / r_size:.4f}, accuracy: {corr / r_size:.4f}')

    acc=corr/len(train_loader.dataset)

    # 평균 손실과 정확도 반
    return r_loss/len(train_loader.dataset), acc



In [ ]:
def test(test_loader,model,loss_fn,device):
    model.eval()

    with torch.no_grad():
        corr=0
        r_loss=0

        for idx,(img,label) in enumerate(test_loader):
            img,label = img.to(device), label.to(device)

            output=model(img)

            _,pred=output.max(dim=1)

            corr+=pred.eq(label).sum().item()

            r_loss+=loss_fn(output,label).item()*img.size(0)

        acc = corr/len(test_loader.dataset)

        return r_loss/len(train_loader.dataset), acc


In [ ]:
min_loss=np.inf

for epoch in range(200):

    train_loss,train_acc = train(train_loader,model,loss_fn,optimizer,device)

    val_loss, val_acc = test(test_loader,model,loss_fn,device)

    #val_loss 가 개선되었다면 min_loss를 갱신하고 model의 가중치(weights)를 저장한다
    if val_loss  < min_loss:
        print('[INFO] val_loss has been improved from {:5f} to {:5f} Saving Model!'.format(min_loss,val_loss))
        min_loss = val_loss
        torch.save(model.state_dict(),'{}.pth'.format('AlexNet'))

    print(f'Epoch:{epoch+1:02d},loss:{train_loss:5f},acc:{train_acc:5f},val_loss:{val_loss:5f},val_acc:{val_acc:5f}')
